In [7]:
import pandas as pd
import pdb
import numpy as np
import itertools
from snakemake.io import expand
import yaml
import os
import sys

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from utils import *
from sm_utils import *

In [9]:
config_file = '../config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [11]:
config_tsv = '../231020_config.tsv'
meta_tsv = '../mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=True)

In [12]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,genotype,sex,age,tissue,humanized,sample,biorep_num,dataset,source,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,...,5xFADHEMI,M,18_months,CTX,False,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,...,5xFADHEMI,M,18_months,CTX,False,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-blk_2,1.0,ad002_10721,10721,...,5xFADWT,M,18_months,CTX,False,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,10721,...,5xFADWT,M,18_months,CTX,False,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12517_lig-blk_1,1.0,ad003_12517,12517,...,5xCLU-h2kbKI_HO,F,4_months,HC,True,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,3


In [15]:
gs = [f for f in df.genotype.unique().tolist()]
for g in gs:
    print(g)

5xFADHEMI
5xFADWT
5xCLU-h2kbKI_HO
CLU-h2kbKI_HO
hTREM2-R47HKI_HET
hTREM2_KI_HET
hTREM2_KI_WT
5xBIN1_HO
BIN1_HO
C57B6J
hABKI_SWE
ABCA7_HO


## hTREM2 pseudochromosome

In [16]:
with open('hTREM2-R47HKI/seq.fa', 'r') as f:
    for line in f:
        if line.startswith('>'):
            continue
        else:
            seq = line.strip()